In [5]:
import pandas as pd
import requests
import time
import os

In [10]:
BASE_URL = "https://musicbrainz.org/ws/2/artist/"
CHECKPOINT_FILE = "./Data/unique_artists_with_country2023.csv"

In [3]:
#"artist"열의 문자열로 쿼리 요청, 응답 데이터 중 country값을 return
def get_artist_country(artist_name):
    headers = {"User-Agent": "Data_Visualization_Practice/1.0 (dhtpals@snu.ac.kr)"}
    params = {
        "query": artist_name,
        "fmt": "json",
    }
    try:
        response = requests.get(BASE_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        if "artists" in data and len(data["artists"]) > 0:
            artist_info = data["artists"][0]
            if "country" in artist_info:
                return artist_info["country"]  
        return "Unknown" 
    except Exception as e:
        print(f"Error fetching data for {artist_name}: {e}")
        return "Error"

In [11]:
if os.path.exists(CHECKPOINT_FILE):
    df = pd.read_csv(CHECKPOINT_FILE)
    print("Checkpoint loaded.")
else:
    df = pd.read_csv("./Data/unique_artists_2023.csv")
    df["country"] = None 

for idx, row in df.iterrows():
    if pd.notna(row["country"]):  
        continue

    artist_name = row["unique_artists"]
    country = get_artist_country(artist_name)
    df.at[idx, "country"] = country

    # 100행마다 중간 저장
    if idx % 100 == 0:
        df.to_csv(CHECKPOINT_FILE, index=False)
        print(f"Checkpoint saved at row {idx}.")

    # API 요청 간 딜레이 부여.
    time.sleep(1)

Checkpoint loaded.


In [12]:
df.to_csv(CHECKPOINT_FILE, index=False)

In [13]:
df.head()

,unique_artists,country
0,Elisa,IT
1,BlackNut,Unknown
2,Manoj Muntashir,IN
3,thasup,IT
4,Fave,Unknown
